In [1]:
import numpy as np
import cv2
import time
import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [6]:
%load_ext autoreload
%autoreload 2

from modules.dataset import Dataset
from modules.detector import FCNDetector
from modules.images_viewer import ImagesViewer
from modules.quality import compute_quality, compute_average_precision, find_optimal_threshold
import modules.models.simple_model
import config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def show(image, size_w=8, size_h=8):
    image = image[:, :, ::-1]
    plt.figure(figsize=(size_w, size_h))
    plt.imshow(image)

# Input

In [4]:
dataset = Dataset()
detector = FCNDetector()

# Show image with rects

# Check NMS

In [ ]:
for image_data in tqdm.tqdm_notebook(dataset.images_data):
    image_data.load()
    mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    if len(image_data.rects) == len(reduced_rects):
        print("{}:\t ok".format(image_data.image_name))
    else:
        print("{}:\t missmatch {} != {}".format(image_data.image_name, len(image_data.rects), len(reduced_rects)))
    image_data.release()

# Check Quality

In [ ]:
quality_objects_union = []
for image_data in tqdm.tqdm_notebook(dataset.images_data):
    image_data.load()
    mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    quality_objects = compute_quality(image_data.rects, reduced_rects)
    quality_objects_union.extend(quality_objects)
    auc_rate = compute_mAP(quality_objects)
    if auc_rate > 0.9999:
        print("{}:\t ok".format(image_data.image_name))
    else:
        print("{}:\t missmatch {}".format(image_data.image_name, auc_rate))

    image_data.release()

In [ ]:
find_optimal_threshold(quality_objects_union)